# Chapter 9: QAOA

---

**Prerequisites:**
- Python 3.8+
- Qiskit 2.1.2
- See `Chapter02_Software.ipynb` for installation instructions


In [8]:
# Setup and imports
import numpy as np
import math
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from IPython.display import display
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import UnitaryGate, MCXGate
from qiskit.circuit.library.standard_gates.u import UGate
from qiskit_aer import Aer
from Chapter09_QAOA_functions import create_qaoa_circuit, qaoa_objective, create_random_graph, estimate_circuit_quality

print('Setup complete!')

Setup complete!


Ex 1

In [9]:
# Solve Max-Cut with QAOA
edges = [(0,1), (1,2), (0,2)]
p = 1  # Single layer

# Initial parameters
params_init = np.random.rand(2*p) * 2*np.pi

# Optimize
result = minimize(qaoa_objective, params_init, 
                  args=(edges, p), method='COBYLA',
                  options={'maxiter': 100})

print(f"Optimal parameters: gamma={result.x[:p]}, beta={result.x[p:]}")
print(f"Minimum energy: {result.fun:.4f}")

# Get final solution distribution
qc_opt = create_qaoa_circuit(result.x[:p], result.x[p:], p)
counts = AerSimulator().run(qc_opt, shots=1000).result().get_counts()

print("\nTop 5 solutions:")
for bitstring, count in sorted(counts.items(), 
                               key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {bitstring}: {count} counts")

NameError: name 'AerSimulator' is not defined

Ex 2

In [ ]:
# Study: QAOA performance vs depth
n_qubits = 6
edges = create_random_graph(n_qubits, edge_prob=0.6)

results = []
for p in range(1, 6):
    params_init = np.random.rand(2*p) * 2*np.pi
    
    result = minimize(qaoa_objective, params_init,
                      args=(edges, p), method='COBYLA',
                      options={'maxiter': 50})
    
    fidelity, depth = estimate_circuit_quality(n_qubits, len(edges), p)
    
    results.append({
        'p': p,
        'energy': result.fun,
        'fidelity': fidelity,
        'depth': depth
    })
    
    print(f"p={p}: energy={result.fun:.4f}, "
          f"fidelity={fidelity:.4f}, depth={depth}")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

p_values = [r['p'] for r in results]
energies = [r['energy'] for r in results]
fidelities = [r['fidelity'] for r in results]

ax1.plot(p_values, energies, 'o-', linewidth=2)
ax1.set_xlabel('Number of layers (p)')
ax1.set_ylabel('Energy (lower is better)')
ax1.set_title('QAOA Solution Quality vs Depth')
ax1.grid(True)

ax2.plot(p_values, fidelities, 'o-', linewidth=2, color='red')
ax2.set_xlabel('Number of layers (p)')
ax2.set_ylabel('Estimated Fidelity')
ax2.set_title('Circuit Fidelity (Chapter 8 Analysis)')
ax2.grid(True)

plt.tight_layout()
plt.savefig('qaoa_scaling.png', dpi=150)